In [191]:
import os
import shutil
import tarfile
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import pandas as pd
import re
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [192]:
train_df = pd.read_csv("task-3-dataset.csv")
train_df["разметка"].loc[train_df["разметка"]=="+"]=1
train_df["разметка"].loc[train_df["разметка"]=="-"]=0
train_df.head()

<ipython-input-192-dc73b64c7d61>:2: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<ipython-input-192-dc73b64c7d61>:3: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignm

,отзывы,разметка
0,Оболочка после чистого андроида тоже очень нек...,0
1,"Нормальный телефон, очень красивая задняя панель",1
2,Деньги на ветер .,0
3,ну так себе,0
4,Ценник вполне адекватный для такой мощной нови...,1


In [193]:
sentiment_counts = train_df['разметка'].value_counts()
fig =px.bar(x= {'0':'Negative','1':'Positive'},
            y= sentiment_counts.values,
            color=sentiment_counts.index,
            color_discrete_sequence =  px.colors.qualitative.Dark24,
            title='<b>Sentiments Counts')

fig.update_layout(title='Sentiments Counts',
                  xaxis_title='Sentiment',
                  yaxis_title='Counts',
                  template='plotly_dark')

# Show the bar chart
fig.show()
pyo.plot(fig, filename = 'Sentiments Counts.html', auto_open = True)

'Sentiments Counts.html'

In [194]:
print(train_df['отзывы'][3])
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
def text_cleaning(review):
    review = review.lower()
    review = review.replace('ё','е')
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^а-я]', ' ', review)
    # Отделяем слова
    review = review.split()
    # Избавляемся от стоп-слов(предлоги,союзы, частицы, не несущие семантической нагрузки)
    review = [word for word in review if not word in set(stopwords.words('russian'))]
    review = ' '.join(review)
    return review
# Train dataset
train_df['отзывы'] = train_df['отзывы'].apply(text_cleaning).tolist()


ну так себе


In [195]:
train_df

,отзывы,разметка
0,оболочка чистого андроида очень некомфортная э...,0
1,нормальный телефон очень красивая задняя панель,1
2,деньги ветер,0
3,,0
4,ценник вполне адекватный мощной новинки неубив...,1
...,...,...
205,короче брать моменты которые доработать,1
206,очень крутой дизайн,1
207,звук ужасный собеседника очень плохо слышно эт...,0
208,подумайте прежде покупать весьма сомнительные ...,0


In [196]:
train_reviews, test_reviews, train_marks, test_marks = train_test_split(train_df['отзывы'], train_df['разметка'], test_size=0.2, random_state=42)

In [197]:

x_val, x_test, y_val, y_test = train_test_split(test_reviews,
                                                    test_marks,
                                                    test_size=0.5,
                                                    stratify = test_marks)

In [198]:
#Tokenize and encode the data using the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, force_download=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [199]:
max_len= 32
# Tokenize and encode the sentences
X_train_encoded = tokenizer.batch_encode_plus(train_reviews.tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

X_val_encoded = tokenizer.batch_encode_plus(x_val.tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')

X_test_encoded = tokenizer.batch_encode_plus(x_test.tolist(),
                                              padding=True,
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')
print(type(X_test_encoded))
print(type(X_val_encoded))
print(type(X_train_encoded))


<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [200]:

k = 0
print('Training Comments -->>',train_reviews[k])
print('\nInput Ids -->>\n',X_train_encoded['input_ids'][k])
print('\nDecoded Ids -->>\n',tokenizer.decode(X_train_encoded['input_ids'][k]))
print('\nAttention Mask -->>\n',X_train_encoded['attention_mask'][k])
print('\nLabels -->>',train_marks[k])

Training Comments -->> оболочка чистого андроида очень некомфортная это дело вкуса привычки

Input Ids -->>
 tf.Tensor(
[  101  1200 25529 10260 22919 10260 15290 22919  1210 16856 23925 14150
 29747 22919 10325  1196 29436 10325 29753 23925 14150 29745  1181 14150
 29436 23742 29753 14150 10325   102     0     0], shape=(32,), dtype=int32)

Decoded Ids -->>
 [CLS] хватает яркости слишком большои [SEP] [PAD] [PAD]

Attention Mask -->>
 tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0], shape=(32,), dtype=int32)

Labels -->> 0


In [201]:
# Intialize the model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [202]:
model.summary()

Model: "tf_bert_for_sequence_classification_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_227 (Dropout)       multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [203]:

# Compile the model with an appropriate optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])



In [204]:
import numpy as np

In [205]:
X = tf.convert_to_tensor(np.hstack([X_train_encoded["input_ids"], X_train_encoded["token_type_ids"], X_train_encoded["attention_mask"]]))

In [206]:
# X = X.reshape((168, 3, 32))
X.shape

TensorShape([168, 96])

In [207]:
X

<tf.Tensor: shape=(168, 96), dtype=int32, numpy=
array([[  101,  1200, 25529, ...,     1,     0,     0],
       [  101,  1200, 14150, ...,     1,     1,     1],
       [  101,  1200, 14150, ...,     1,     1,     1],
       ...,
       [  101,  1187, 10260, ...,     1,     1,     1],
       [  101,  1196, 10260, ...,     1,     1,     1],
       [  101,  1192, 29748, ...,     1,     1,     1]], dtype=int32)>

In [208]:
print(np.hstack([X_val_encoded['input_ids'], X_val_encoded['token_type_ids'], X_val_encoded['attention_mask']]).shape)

(21, 96)


In [209]:
np.asarray(y_val).shape

(21,)

In [240]:
import numpy as np
# Step 5: Train the model
print(train_marks.shape)
TM = tf.convert_to_tensor(np.asarray(train_marks).astype("int"))

print(TM.shape)

history = model.fit(X,
    TM,
    validation_data=(
      tf.convert_to_tensor(np.hstack([X_val_encoded['input_ids'], X_val_encoded['token_type_ids'], X_val_encoded['attention_mask']]).astype("int")),tf.convert_to_tensor(np.asarray(y_val).astype("int"))),
    batch_size=32,
    epochs=12
)

(168,)
(168,)
Epoch 1/12
6/6 [==============================] - 4s 578ms/step - loss: 0.0619 - accuracy: 0.9940 - val_loss: 0.8549 - val_accuracy: 0.8095
Epoch 2/12
6/6 [==============================] - 4s 585ms/step - loss: 0.0476 - accuracy: 0.9881 - val_loss: 0.7636 - val_accuracy: 0.8095
Epoch 3/12
6/6 [==============================] - 4s 585ms/step - loss: 0.0432 - accuracy: 0.9940 - val_loss: 0.9196 - val_accuracy: 0.7619
Epoch 4/12
6/6 [==============================] - 4s 591ms/step - loss: 0.0487 - accuracy: 0.9881 - val_loss: 0.9737 - val_accuracy: 0.7619
Epoch 5/12
6/6 [==============================] - 4s 593ms/step - loss: 0.0435 - accuracy: 0.9940 - val_loss: 1.1733 - val_accuracy: 0.7143
Epoch 6/12
6/6 [==============================] - 4s 644ms/step - loss: 0.0331 - accuracy: 0.9940 - val_loss: 1.2573 - val_accuracy: 0.7143
Epoch 7/12
6/6 [==============================] - 4s 612ms/step - loss: 0.0393 - accuracy: 0.9881 - val_loss: 1.2207 - val_accuracy: 0.7143
Epoch 

In [242]:
path = '/content'
# Save tokenizer
tokenizer.save_pretrained(path +'/Tokenizer')

# Save model
model.save_pretrained(path +'/Model')

In [225]:
X_test = np.hstack([X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']]).astype("int")

In [226]:
X_test.shape

(21, 96)

In [233]:
y_test = np.asarray(y_test).astype("int")

In [231]:
type(y_test)

numpy.ndarray

In [243]:
test_loss, test_accuracy = model.evaluate(
    X_test,
    y_test
)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

1/1 [==============================] - 0s 219ms/step - loss: 1.4185 - accuracy: 0.7143
Test loss: 1.4184739589691162, Test accuracy: 0.7142857313156128


In [211]:
X_train = pad_sequences(X_train_token, maxlen=maxlen, padding='post')
X_test  = pad_sequences(X_test_token, maxlen=maxlen, padding='post')

NameError: name 'pad_sequences' is not defined